In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2

labels = os.listdir('train')
labels

['Open', 'no_yawn', 'yawn', 'Closed']

In [5]:
from tqdm import tqdm
def get_yawn_data(direc="train"):
    yaw_no = []
    IMG_SIZE = 100
    categories = ["yawn", "no_yawn"]
    for category in categories:
        path_link = os.path.join(direc, category)
        class_num1 = categories.index(category)
        print(class_num1)
        for image in tqdm(os.listdir(path_link)):
            image_array = cv2.imread(os.path.join(path_link, image), cv2.IMREAD_COLOR)
            face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
            for (x, y, w, h) in faces:
                img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                roi_color = img[y:y+h, x:x+w]
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                yaw_no.append([resized_array, class_num1])
    return yaw_no

data = get_yawn_data()
X,y = list(zip(*data))

  1%|          | 4/723 [00:00<00:20, 34.54it/s]

0


  1%|          | 4/725 [00:00<00:20, 35.32it/s]

1


100%|██████████| 725/725 [00:20<00:00, 35.53it/s]


In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

X = np.array(X)
X = X.reshape(-1, 100, 100, 3)
y = LabelBinarizer().fit_transform(y)
y = np.array(y)

test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [7]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

train_generator = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
test_generator = ImageDataGenerator(rescale=1/255)

train_generator = train_generator.flow(np.array(X_train), y_train, shuffle=False)
test_generator = test_generator.flow(np.array(X_test), y_test, shuffle=False)

In [16]:
model = Sequential()

model.add(Conv2D(256, (3, 3), activation="relu", input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.7))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 98, 98, 256)       7168      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 49, 49, 256)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 47, 47, 128)       295040    
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 21, 21, 64)        73792     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 8, 8, 32)         

In [17]:
history = model.fit(train_generator, epochs=50, validation_data=test_generator, shuffle=True, validation_steps=len(test_generator))

Epoch 1/50
11/11 [==============================] - 1s 63ms/step - loss: 0.7040 - accuracy: 0.4773 - val_loss: 0.6978 - val_accuracy: 0.4577
Epoch 2/50
11/11 [==============================] - 1s 53ms/step - loss: 0.6943 - accuracy: 0.5317 - val_loss: 0.6929 - val_accuracy: 0.4577
Epoch 3/50
11/11 [==============================] - 1s 53ms/step - loss: 0.6850 - accuracy: 0.5861 - val_loss: 0.7328 - val_accuracy: 0.4577
Epoch 4/50
11/11 [==============================] - 1s 53ms/step - loss: 0.6365 - accuracy: 0.6193 - val_loss: 0.6387 - val_accuracy: 0.6408
Epoch 5/50
11/11 [==============================] - 1s 53ms/step - loss: 0.5536 - accuracy: 0.7432 - val_loss: 0.7500 - val_accuracy: 0.6972
Epoch 6/50
11/11 [==============================] - 1s 56ms/step - loss: 0.6023 - accuracy: 0.7190 - val_loss: 0.6361 - val_accuracy: 0.6972
Epoch 7/50
11/11 [==============================] - 1s 53ms/step - loss: 0.5321 - accuracy: 0.6798 - val_loss: 0.6989 - val_accuracy: 0.7042
Epoch 8/50
11

In [18]:
from sklearn.metrics import classification_report

labels_new = ["yawn", "no_yawn"]
prediction = model.predict_classes(X_test)
print(classification_report(y_test, prediction, target_names=labels_new))

              precision    recall  f1-score   support

        yawn       0.97      0.88      0.92        65
     no_yawn       0.90      0.97      0.94        77

    accuracy                           0.93       142
   macro avg       0.93      0.93      0.93       142
weighted avg       0.93      0.93      0.93       142



In [19]:
model.save("yawn.model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: yawn.model/assets


In [20]:
model.save("yawn.h5")